# Library importation

In [1]:
import numpy as np
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import re
import ast 
import json

# Recupération de la liste des brevets marqués

In [2]:
list_patents = pd.read_csv('./Data/patents_number.csv',index_col=None)

In [3]:
L = list(list_patents.patent_number.values)

In [85]:
df = pd.DataFrame(columns=['patent_number', 'patent_date',
                           'patent_processing_time', 'ipc_section', 'ipc_class', 'assignee_id', ])

# Récuperation des informations des brevets marqués

In [5]:
i=0
while i < (len(L)-700):    
    token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
    token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + str(L[i:i+700]) + token2
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["patents"]
    df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
    i+=700
    print("{}% done".format(i/len(L)*100))
token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
url = token1 + str(L[i:]) + token2
r = requests.get(url)
html=r.text
html = html.replace("null","None")
Dict = ast.literal_eval(html)["patents"]
df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
df = df.reset_index(drop=True)

2.8957928267074835% done
5.791585653414967% done
8.687378480122451% done
11.583171306829934% done
14.478964133537417% done
17.374756960244902% done
20.270549786952387% done
23.166342613659868% done
26.06213544036735% done
28.957928267074834% done
31.853721093782315% done
34.749513920489804% done
37.64530674719729% done
40.54109957390477% done
43.43689240061225% done
46.332685227319736% done
49.22847805402722% done
52.1242708807347% done
55.02006370744219% done
57.91585653414967% done
60.81164936085715% done
63.70744218756463% done
66.60323501427212% done
69.49902784097961% done
72.39482066768709% done
75.29061349439458% done
78.18640632110206% done
81.08219914780955% done
83.97799197451702% done
86.8737848012245% done
89.769577627932% done
92.66537045463947% done
95.56116328134696% done
98.45695610805444% done


In [6]:
df["marked"]=[1]*len(df)

In [7]:
assignee_id=[]

for index,j in enumerate(df.assignees.values):
    assignees=[]
    for element in j:
        assignees.append(element["assignee_id"])
    if len(assignees)>1:
        assignee_id.append(assignees)
    else:
        assignee_id.append(assignees[0])
df["assignee_id"] = assignee_id
df = df.drop(['assignees'],axis=1)

In [8]:
df

,patent_number,patent_date,patent_processing_time,ipc_section,ipc_class,assignee_id,IPCs,wipos,marked
0,5387603,1995-02-07,433,NaN,NaN,5330279e-a21e-438c-80d1-7e6ae793c092,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
1,5771295,1998-06-23,552,NaN,NaN,2986e46e-7937-477f-9bc5-1f5eec05183f,"[{'ipc_section': 'H', 'ipc_class': '04'}]",[{'wipo_field_id': '2'}],1
2,5585587,1996-12-17,832,NaN,NaN,f7152515-d420-42d9-a7b6-3a2796bc5d10,"[{'ipc_section': 'G', 'ipc_class': '10'}]","[{'wipo_field_id': '34'}, {'wipo_field_id': '2...",1
3,5321490,1994-06-14,568,NaN,NaN,ec72750e-f789-4749-91f7-6292f7c6fda7,"[{'ipc_section': 'G', 'ipc_class': '01'}]","[{'wipo_field_id': '10'}, {'wipo_field_id': '1...",1
4,5750497,1998-05-12,1161,NaN,NaN,9a3afa40-66e5-4921-b831-64287f2052e0,"[{'ipc_section': 'C', 'ipc_class': '07'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
...,...,...,...,...,...,...,...,...,...
23905,10299691,2019-05-28,225,NaN,NaN,1b2d67d2-f1f9-4c29-8cd1-a5e75ba27be7,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...",[{'wipo_field_id': '13'}],1
23906,10292890,2019-05-21,966,NaN,NaN,f0997ba8-1ec2-4c0c-aaea-d80066a49317,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1
23907,10299968,2019-05-28,343,NaN,NaN,e5fc32c3-a913-455a-94ce-9932d30eefff,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '13'}, {'wipo_field_id': '6'}]",1
23908,10149616,2018-12-11,2133,NaN,NaN,4eb39673-7f59-4c45-bf6c-2275dccf9048,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1


# Constructing the list of assignees

In [9]:
assignees = []
for el in df.assignee_id:
    if type(el)==list:
        for i in el:
            if i not in assignees:
                assignees.append(i)
    elif el!=None and el not in assignees:
        assignees.append(el)

In [10]:
L_assignees = []
k=0
while k < len(assignees)-100:
    L_assignees.append(json.dumps(assignees[k:k+100]))
    k+=100
L_assignees.append(json.dumps(assignees[k:]))

# Récuperation des brevets non marqués de ces entreprises 

In [11]:
L_unmarked=[]
#df_list_assignee = pd.DataFrame()
for i, el in enumerate(L_assignees):
    token1 = 'https://api.patentsview.org/assignees/query?q={"assignee_id":'
    token2 = '}&f=["patent_number","assignee_id"]&o={"page":'
    token3 = ',"per_page":10000}'
    url = token1 + el + token2 + str(1) + token3
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["assignees"]
    for d in Dict:
        patents = d["patents"]
        for patent in patents:
            if patent["patent_number"].isnumeric():
                L_unmarked.append(int(patent["patent_number"]))
            else:
                L_unmarked.append("{}".format(patent["patent_number"]))
    #     df_list_assignee = pd.concat([df_list_assignee,pd.read_json(json.dumps(Dict),orient='records')])
    print(i*100)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [12]:
L_unmarked = list(set(L_unmarked)-set(df.patent_number.values))

In [13]:
len(L_unmarked)

1893036

In [14]:
i=0
while i < (len(L_unmarked)-600):    
    token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
    token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + str(L_unmarked[i:i+600]) + token2
    url = url.replace("\'",'"')
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["patents"]
    df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
    i+=600
    print("{}% done".format(i/len(L_unmarked)*100))
token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
url = token1 + str(L_unmarked[i:]) + token2
r = requests.get(url)
html=r.text
html = html.replace("null","None")
Dict = ast.literal_eval(html)["patents"]
df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
df = df.reset_index(drop=True)

0.03169511831787668% done
0.06339023663575336% done
0.09508535495363005% done
0.1267804732715067% done
0.1584755915893834% done
0.1901707099072601% done
0.22186582822513679% done
0.2535609465430134% done
0.2852560648608901% done
0.3169511831787668% done
0.3486463014966435% done
0.3803414198145202% done
0.4120365381323969% done
0.44373165645027357% done
0.4754267747681502% done
0.5071218930860268% done
0.5388170114039036% done
0.5705121297217802% done
0.602207248039657% done
0.6339023663575336% done
0.6655974846754102% done
0.697292602993287% done
0.7289877213111636% done
0.7606828396290404% done
0.7923779579469169% done
0.8240730762647938% done
0.8557681945826704% done
0.8874633129005471% done
0.9191584312184238% done
0.9508535495363004% done
0.982548667854177% done
1.0142437861720537% done
1.0459389044899303% done
1.0776340228078072% done
1.1093291411256838% done
1.1410242594435605% done
1.1727193777614373% done
1.204414496079314% done
1.2361096143971906% done
1.2678047327150672% done

10.712949991442319% done
10.744645109760196% done
10.776340228078071% done
10.808035346395949% done
10.839730464713824% done
10.871425583031701% done
10.903120701349579% done
10.934815819667456% done
10.966510937985332% done
10.998206056303209% done
11.029901174621084% done
11.061596292938962% done
11.093291411256837% done
11.124986529574715% done
11.156681647892592% done
11.18837676621047% done
11.220071884528345% done
11.251767002846222% done
11.283462121164098% done
11.315157239481975% done
11.34685235779985% done
11.378547476117728% done
11.410242594435605% done
11.441937712753482% done
11.473632831071358% done
11.505327949389235% done
11.53702306770711% done
11.568718186024988% done
11.600413304342865% done
11.632108422660743% done
11.663803540978618% done
11.695498659296495% done
11.72719377761437% done
11.758888895932248% done
11.790584014250125% done
11.822279132568001% done
11.85397425088588% done
11.885669369203756% done
11.917364487521633% done
11.949059605839508% done
11.98

21.235729272977377% done
21.267424391295254% done
21.299119509613128% done
21.330814627931005% done
21.362509746248882% done
21.39420486456676% done
21.425899982884637% done
21.45759510120251% done
21.48929021952039% done
21.520985337838265% done
21.552680456156143% done
21.58437557447402% done
21.616070692791897% done
21.647765811109775% done
21.67946092942765% done
21.711156047745526% done
21.742851166063403% done
21.77454628438128% done
21.806241402699158% done
21.83793652101703% done
21.869631639334912% done
21.901326757652786% done
21.933021875970663% done
21.964716994288537% done
21.996412112606418% done
22.02810723092429% done
22.05980234924217% done
22.091497467560046% done
22.123192585877923% done
22.1548877041958% done
22.186582822513675% done
22.218277940831555% done
22.24997305914943% done
22.281668177467306% done
22.313363295785184% done
22.34505841410306% done
22.37675353242094% done
22.408448650738812% done
22.44014376905669% done
22.471838887374567% done
22.503534005692

31.82189879114819% done
31.853593909466067% done
31.88528902778394% done
31.916984146101818% done
31.948679264419695% done
31.98037438273757% done
32.01206950105544% done
32.04376461937332% done
32.075459737691205% done
32.10715485600908% done
32.13884997432695% done
32.17054509264483% done
32.20224021096271% done
32.233935329280584% done
32.26563044759846% done
32.29732556591634% done
32.329020684234216% done
32.36071580255209% done
32.39241092086997% done
32.42410603918785% done
32.45580115750572% done
32.487496275823595% done
32.51919139414147% done
32.55088651245935% done
32.58258163077723% done
32.614276749095104% done
32.64597186741298% done
32.67766698573086% done
32.709362104048736% done
32.74105722236661% done
32.772752340684484% done
32.80444745900237% done
32.836142577320246% done
32.867837695638116% done
32.89953281395599% done
32.93122793227387% done
32.96292305059175% done
32.994618168909625% done
33.0263132872275% done
33.05800840554538% done
33.08970352386326% done
33.1

42.566543900908385% done
42.598239019226256% done
42.62993413754413% done
42.66162925586201% done
42.69332437417989% done
42.725019492497765% done
42.75671461081564% done
42.78840972913352% done
42.8201048474514% done
42.851799965769274% done
42.883495084087144% done
42.91519020240502% done
42.9468853207229% done
42.97858043904078% done
43.01027555735865% done
43.04197067567653% done
43.07366579399441% done
43.105360912312285% done
43.13705603063016% done
43.16875114894804% done
43.20044626726592% done
43.232141385583795% done
43.26383650390167% done
43.29553162221955% done
43.32722674053742% done
43.3589218588553% done
43.390616977173174% done
43.42231209549105% done
43.45400721380893% done
43.485702332126806% done
43.51739745044468% done
43.54909256876256% done
43.58078768708044% done
43.612482805398315% done
43.644177923716185% done
43.67587304203406% done
43.70756816035195% done
43.739263278669824% done
43.770958396987695% done
43.80265351530557% done
43.83434863362345% done
43.866

...

64.02413900211089% done
64.05583412042877% done
64.08752923874664% done
64.11922435706452% done
64.15091947538241% done
64.18261459370028% done
64.21430971201816% done
64.24600483033603% done
64.2776999486539% done
64.30939506697179% done
64.34109018528966% done
64.37278530360754% done
64.40448042192541% done
64.43617554024328% done
64.46787065856117% done
64.49956577687904% done
64.53126089519692% done
64.56295601351479% done
64.59465113183268% done
64.62634625015056% done
64.65804136846843% done
64.6897364867863% done
64.72143160510419% done
64.75312672342206% done
64.78482184173994% done
64.81651696005781% done
64.8482120783757% done
64.87990719669357% done
64.91160231501144% done
64.94329743332932% done
64.97499255164719% done
65.00668766996507% done
65.03838278828295% done
65.07007790660082% done
65.1017730249187% done
65.13346814323658% done
65.16516326155445% done
65.19685837987234% done
65.22855349819021% done
65.2602486165081% done
65.29194373482596% done
65.32363885314383% do

74.92725970346046% done
74.95895482177835% done
74.99064994009622% done
75.0223450584141% done
75.05404017673199% done
75.08573529504986% done
75.11743041336774% done
75.14912553168561% done
75.18082065000348% done
75.21251576832137% done
75.24421088663924% done
75.27590600495712% done
75.30760112327499% done
75.33929624159286% done
75.37099135991075% done
75.40268647822862% done
75.4343815965465% done
75.46607671486437% done
75.49777183318224% done
75.52946695150014% done
75.56116206981801% done
75.59285718813588% done
75.62455230645377% done
75.65624742477164% done
75.68794254308952% done
75.71963766140739% done
75.75133277972526% done
75.78302789804314% done
75.81472301636101% done
75.8464181346789% done
75.87811325299677% done
75.90980837131465% done
75.94150348963252% done
75.9731986079504% done
76.00489372626828% done
76.03658884458616% done
76.06828396290403% done
76.09997908122192% done
76.13167419953979% done
76.16336931785767% done
76.19506443617554% done
76.22675955449341% d

85.83038040481004% done
85.86207552312793% done
85.8937706414458% done
85.92546575976368% done
85.95716087808157% done
85.98885599639944% done
86.0205511147173% done
86.05224623303519% done
86.08394135135306% done
86.11563646967095% done
86.14733158798882% done
86.1790267063067% done
86.21072182462457% done
86.24241694294244% done
86.27411206126033% done
86.3058071795782% done
86.33750229789608% done
86.36919741621395% done
86.40089253453183% done
86.43258765284972% done
86.46428277116759% done
86.49597788948546% done
86.52767300780334% done
86.55936812612121% done
86.5910632444391% done
86.62275836275697% done
86.65445348107484% done
86.68614859939272% done
86.7178437177106% done
86.74953883602848% done
86.78123395434635% done
86.81292907266422% done
86.8446241909821% done
86.87631930929997% done
86.90801442761786% done
86.93970954593574% done
86.97140466425361% done
87.0030997825715% done
87.03479490088937% done
87.06649001920724% done
87.09818513752512% done
87.12988025584299% done


96.73350110615962% done
96.7651962244775% done
96.79689134279539% done
96.82858646111326% done
96.86028157943115% done
96.89197669774902% done
96.92367181606689% done
96.95536693438477% done
96.98706205270264% done
97.01875717102052% done
97.0504522893384% done
97.08214740765627% done
97.11384252597415% done
97.14553764429202% done
97.1772327626099% done
97.20892788092777% done
97.24062299924566% done
97.27231811756353% done
97.30401323588141% done
97.33570835419928% done
97.36740347251717% done
97.39909859083504% done
97.43079370915292% done
97.46248882747079% done
97.49418394578868% done
97.52587906410655% done
97.55757418242442% done
97.5892693007423% done
97.62096441906017% done
97.65265953737806% done
97.68435465569593% done
97.7160497740138% done
97.74774489233168% done
97.77944001064955% done
97.81113512896744% done
97.84283024728532% done
97.87452536560319% done
97.90622048392108% done
97.93791560223895% done
97.96961072055682% done
98.0013058388747% done
98.03300095719257% don

KeyError: 'patents'

In [15]:
df = df.reset_index(drop=True)

In [28]:
df.to_csv('./to_clean.csv',index=False)

In [19]:
df.patent_number = df.patent_number.astype(str)

In [31]:
df.marked = df.marked.fillna(0)

In [32]:
df

,patent_number,patent_date,patent_processing_time,ipc_section,ipc_class,assignee_id,IPCs,wipos,marked,assignees
0,5387603,1995-02-07,433,NaN,NaN,5330279e-a21e-438c-80d1-7e6ae793c092,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1.0,NaN
1,5771295,1998-06-23,552,NaN,NaN,2986e46e-7937-477f-9bc5-1f5eec05183f,"[{'ipc_section': 'H', 'ipc_class': '04'}]",[{'wipo_field_id': '2'}],1.0,NaN
2,5585587,1996-12-17,832,NaN,NaN,f7152515-d420-42d9-a7b6-3a2796bc5d10,"[{'ipc_section': 'G', 'ipc_class': '10'}]","[{'wipo_field_id': '34'}, {'wipo_field_id': '2...",1.0,NaN
3,5321490,1994-06-14,568,NaN,NaN,ec72750e-f789-4749-91f7-6292f7c6fda7,"[{'ipc_section': 'G', 'ipc_class': '01'}]","[{'wipo_field_id': '10'}, {'wipo_field_id': '1...",1.0,NaN
4,5750497,1998-05-12,1161,NaN,NaN,9a3afa40-66e5-4921-b831-64287f2052e0,"[{'ipc_section': 'C', 'ipc_class': '07'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1916306,8387852,2013-03-05,1970-01-01 00:43:32,NaN,NaN,NaN,"[{'ipc_section': 'B', 'ipc_class': '23'}]",[{'wipo_field_id': '26'}],0.0,[{'assignee_id': '4797190e-68c2-4d9d-9b8d-e6c1...
1916307,8387016,2013-02-26,1970-01-01 00:23:17,NaN,NaN,NaN,"[{'ipc_section': 'G', 'ipc_class': '06'}]",[{'wipo_field_id': '6'}],0.0,[{'assignee_id': '4674eaf4-2927-46ab-ae18-9d92...
1916308,8387331,2013-03-05,1970-01-01 00:10:20,NaN,NaN,NaN,"[{'ipc_section': 'E', 'ipc_class': '04'}, {'ip...","[{'wipo_field_id': '27'}, {'wipo_field_id': '3...",0.0,[{'assignee_id': 'f2d3dc32-b29c-4c55-ab6d-736b...
1916309,8388526,2013-03-05,1970-01-01 00:29:37,NaN,NaN,NaN,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],0.0,[{'assignee_id': 'f39637f8-f2d8-4b4a-a5ee-9b26...


# Nettoyage du DataFrame

In [44]:
ipc_section=[]
for index,j in enumerate(df.IPCs.values):
    tmp_sec=[]
    ipc = j
    for ele in ipc:
        tmp_sec.append(ele["ipc_section"])
    ipc_section.append(list(np.unique(tmp_sec)))
    if index%100000==0:
        print(index)
df.ipc_section=ipc_section

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [46]:
df = df.drop(['IPCs'],axis=1)

In [57]:
df = df.drop(['ipc_class'],axis=1)

In [81]:
assignees_unmarked=[]
for i,el in enumerate(df[df.marked==0].assignees.values):
    tmp_assignees=[]
    for dictionnary in el:
        if dictionnary['assignee_id'] in assignees:
            tmp_assignees.append(dictionnary['assignee_id'])
    assignees_unmarked.append(tmp_assignees)
df.assignee_id = (list(df[df.marked==1].assignee_id.values)).extend(assignees_unmarked)

In [87]:
df_final

,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,assignees
0,5387603,1995-02-07,433,"[A, C]",None,"[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1.0,NaN
1,5771295,1998-06-23,552,[H],None,[{'wipo_field_id': '2'}],1.0,NaN
2,5585587,1996-12-17,832,[G],None,"[{'wipo_field_id': '34'}, {'wipo_field_id': '2...",1.0,NaN
3,5321490,1994-06-14,568,[G],None,"[{'wipo_field_id': '10'}, {'wipo_field_id': '1...",1.0,NaN
4,5750497,1998-05-12,1161,"[A, C]",None,"[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1.0,NaN
...,...,...,...,...,...,...,...,...
1915706,D555179,2007-11-13,299,[None],None,[{'wipo_field_id': None}],0.0,[{'assignee_id': 'dc77a1b1-f087-4a19-992d-6393...
1915707,4192319,1980-03-11,893,[A],None,[{'wipo_field_id': '13'}],0.0,[{'assignee_id': 'fa543ac8-5ee8-4999-b949-6c8a...
1915708,8386739,2013-02-26,1247,[G],None,[{'wipo_field_id': '6'}],0.0,[{'assignee_id': 'ee0168d9-bf74-4513-bfbf-e73c...
1915709,4191891,1980-03-04,481,[H],None,[{'wipo_field_id': '13'}],0.0,[{'assignee_id': 'f2d3dc32-b29c-4c55-ab6d-736b...


In [88]:
df_final.to_csv('./to_finish.csv',index=False)

# Assignees df

In [75]:
df_assignees=pd.DataFrame(columns=["assignee_id", "assignee_organization","assignee_type", "assignee_lastknown_country", "assignee_total_num_patents", "assignee_total_num_inventors"])

In [76]:
#L_tmp = list(np.unique(df.assignee_id.dropna().values))
L2 = []
k=0
while k < len(assignees)-100:
    L2.append(json.dumps(assignees[k:k+100]))
    k+=100
L2.append(json.dumps(assignees[k:]))

In [77]:
for i in range(len(L2)):    
    token1 ='https://api.patentsview.org/assignees/query?q={"assignee_id":'
    token2 = '}&f=["assignee_id","assignee_organization","assignee_type","assignee_lastknown_country", "assignee_total_num_patents", "assignee_total_num_inventors"]&s=[{"assignee_organization":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + L2[i] + token2
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["assignees"]
    df_assignees = pd.concat([df_assignees,pd.read_json(json.dumps(Dict),orient='records')])
    print(i*100)
df_assignees=df_assignees.reset_index(drop=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [80]:
np.argmax(df_assignees.assignee_total_num_patents.values)

153

In [ ]:
df.to_csv('./info_patents.csv')
df_assignees.to_csv('./info_companies.csv')

In [82]:
df_assignees.iloc[153]

assignee_id                            ee0168d9-bf74-4513-bfbf-e73c933f90e7
assignee_organization           International Business Machines Corporation
assignee_type                                                             2
assignee_lastknown_country                                               US
assignee_total_num_patents                                           142160
assignee_total_num_inventors                                          63301
Name: 153, dtype: object